In [ ]:
!bash drive/My\ Drive/datawithcred.sh

In [ ]:
from pathlib import Path
import tensorflow as tf
from src.data.make_dataset import GetDataset
from src.core.cloud import CloudTrainer

In [ ]:
tf.random.set_seed(111)

In [ ]:
dt = GetDataset(image_dims=(224, 224))

In [ ]:
train = dt.make_train_data(Path("chest_xray/train"))

In [ ]:
val = dt.make_train_data(Path("chest_xray/test"))

In [ ]:
trainer = CloudTrainer(experiment_name="LeNet5", save_freq=50)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=./drive/My\ Drive/models --port=6006

In [ ]:
def get_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(dt.width, dt.height, 3)),
        tf.keras.layers.Conv2D(filters=6, kernel_size=(5,5), activation="relu"),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),
        tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation="relu"),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),
        tf.keras.layers.Conv2D(filters=120, kernel_size=(5,5), activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation="relu"),
        tf.keras.layers.Dense(84, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    return model

In [ ]:
def get_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(dt.width, dt.height, 3)),
        tf.keras.layers.Conv2D(filters=6, kernel_size=(5,5), activation="tanh"),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),
        tf.keras.layers.Conv2D(filters=16, kernel_size=(5, 5), activation="tanh"),
        tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)),
        tf.keras.layers.Conv2D(filters=120, kernel_size=(5,5), activation="tanh"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation="tanh"),
        tf.keras.layers.Dense(84, activation="tanh"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    return model

In [ ]:
trainer.add_keras_seq_model(get_model)

In [ ]:
trainer.add_callbacks([tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=10)])

In [ ]:
trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy",
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')
             ]
)

In [ ]:
trainer.fit(
    train,
    validation_data=val,
    epochs=70
)

In [ ]:
trainer.model.save("final_model.h5")

In [ ]:
!mv final_model.h5 drive/

In [ ]:
trainer.model.summary()

In [ ]:
aug_trainer = CloudTrainer(experiment_name="LeNet5-balanced", save_freq=50)

In [ ]:
train = dt.make_balaced_dataset(majority_class="chest_xray/train/PNEUMONIA",
                                minority_class="chest_xray/train/NORMAL",
                                transformations=[
                                    dt.image_flip_left_right,
                                    dt.image_central_crop,
                                    dt.image_adjust_saturation
                                ])

In [ ]:
aug_trainer.add_keras_seq_model(get_model)

In [ ]:
trainer.add_callbacks([tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=10, verbose=1)])

In [ ]:
aug_trainer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy",
             tf.keras.metrics.Precision(name='precision'),
             tf.keras.metrics.Recall(name='recall')]
)

In [ ]:
aug_trainer.fit(
    train,
    validation_data=val,
    epochs=70
)